In [1]:
import numpy as np
from numpy import random

def inverse_cdf(su, W):
    """
    Inverse CDF Algorithm for a finite distribution
    
    Parameters
    -----------------
    su : (M,) ndarray
        M sorted variates (i.e. M ordered points in [0,1)
    W: (N,) ndarray
        a vector of N normalized weights (>=0 and sum to one)
    
    """
    j = 0
    s = W[0]
    M = su.shape[0]
    A = np.empty(M, 'int')
    for n in range(M):
        while su[n] > s:
            j += 1
            s += W[j]
        A[n] = j
    return A


def uniform_spacings(N):
    z = np.cumsum(-np.log(random.rand(N+1)))
    return z[:-1]/z[-1]


def multinomial(W, M):
    return inverse_cdf(uniform_spacings(M), W)


def multinomial_once(W):
    """ Sample once from a Multinomial distribution
    Parameters
    ----------
    W: (N,) ndarray
        normalized weights (>=0, sum to one)
    Returns
    -------
    int
        a single draw from the discrete distribution that generates n with
        probability W[n]
    Note
    ----
    This is equivalent to
       A = multinomial(W, M=1)
    but it is faster.
    """
    return np.searchsorted(np.cumsum(W), random.rand())

In [2]:
w = np.array([0.1,0.25, 0.65])
multinomial(w, 10)

array([0, 0, 1, 1, 1, 2, 2, 2, 2, 2])

In [3]:
multinomial_once(w)

2

In [4]:
letters = np.array(['a', 'b', 'c'])
letters[multinomial(w, 10)]

array(['a', 'b', 'b', 'c', 'c', 'c', 'c', 'c', 'c', 'c'], dtype='<U1')

In [5]:
import altair as alt
import pandas as pd

### My scheme
source = pd.DataFrame(
    random.choice(
        ['a','b','c'],
        p = w,
        replace=True,
        size=1000
    ),
    columns=['letter']
)

source

alt.Chart(source).mark_bar().encode(
    alt.Y("letter"),
    x='count()'
)


alt.Chart(...)

In [6]:

### Chopin's scheme

letters = np.array(['a', 'b', 'c'])

source = pd.DataFrame(
    letters[multinomial(w, 1000)],
    columns=['letter']
)

alt.Chart(source).mark_bar().encode(
    alt.Y("letter"),
    x='count()'
)


alt.Chart(...)

In [7]:
w = np.arange(10)
nw = w / np.sum(w).astype(float)
nw

array([0.        , 0.02222222, 0.04444444, 0.06666667, 0.08888889,
       0.11111111, 0.13333333, 0.15555556, 0.17777778, 0.2       ])

In [15]:
np.testing.assert_allclose(1., nw.sum())

In [8]:
# Chopin's particle resampling 

ps = dict()
ps['M'] = 10
ps['param_names'] = ['a', 'b']
ps['a'] = np.arange(10.)
ps['b'] = np.arange(10.,20.)

indx = multinomial(nw, ps['M'])
ps['a'] = ps['a'][indx]
ps['b'] = ps['b'][indx]

In [9]:
import timeit 

mysetup = '''
import numpy as np
from numpy import random

def inverse_cdf(su, W):
    """
    Inverse CDF Algorithm for a finite distribution
    
    Parameters
    -----------------
    su : (M,) ndarray
        M sorted variates (i.e. M ordered points in [0,1)
    W: (N,) ndarray
        a vector of N normalized weights (>=0 and sum to one)
    
    """
    j = 0
    s = W[0]
    M = su.shape[0]
    A = np.empty(M, 'int')
    for n in range(M):
        while su[n] > s:
            j += 1
            s += W[j]
        A[n] = j
    return A


def uniform_spacings(N):
    z = np.cumsum(-np.log(random.rand(N+1)))
    return z[:-1]/z[-1]


def multinomial(W, M):
    return inverse_cdf(uniform_spacings(M), W)


def multinomial_once(W):
    """ Sample once from a Multinomial distribution
    Parameters
    ----------
    W: (N,) ndarray
        normalized weights (>=0, sum to one)
    Returns
    -------
    int
        a single draw from the discrete distribution that generates n with
        probability W[n]
    Note
    ----
    This is equivalent to
       A = multinomial(W, M=1)
    but it is faster.
    """
    return np.searchsorted(np.cumsum(W), random.rand())



w = np.arange(10)
nw = w / np.sum(w).astype(float)

ps = dict()
ps['M'] = 10
ps['param_names'] = ['a', 'b']
ps['a'] = np.arange(10)
ps['b'] = np.arange(10,20)
'''

mycode = '''
indx = multinomial(nw, ps['M'])
ps['a'] = ps['a'][indx]
ps['b'] = ps['b'][indx]
'''


# timeit statement 
print(timeit.timeit(setup = mysetup, 
                    stmt = mycode, 
                    number = 10000))

0.14979355596005917


In [10]:
# My particle resampling 


from codebase.ibis import multinomial_sample_particles
ps = dict()
ps['M'] = 10
ps['param_names'] = ['a', 'b']
ps['a'] = np.arange(10)
ps['b'] = np.arange(10,20)

multinomial_sample_particles(ps, nw)




{'M': 10,
 'param_names': ['a', 'b'],
 'a': array([7, 9, 9, 5, 6, 7, 6, 5, 4, 3]),
 'b': array([17, 19, 19, 15, 16, 17, 16, 15, 14, 13])}

In [11]:
import timeit 

mysetup = '''
from codebase.ibis import multinomial_sample_particles
import numpy as np


w = np.arange(10)
nw = w / np.sum(w).astype(float)

ps = dict()
ps['M'] = 10
ps['param_names'] = ['a', 'b']
ps['a'] = np.arange(10)
ps['b'] = np.arange(10,20)
'''

mycode = '''
multinomial_sample_particles(ps, w)
'''


# timeit statement 
print(timeit.timeit(setup = mysetup, 
                    stmt = mycode, 
                    number = 10000))

0.31398043199442327
